In [59]:
import os
import zipfile
import pandas as pd
import json
import ast
import gc
import re
import numpy as np

In [60]:
from datetime import datetime, timedelta
Date = datetime.today().strftime('%d%m%Y')

In [61]:
!pip install mysql-connector-python
!pip install pysqlite3
!curl ifconfig.me

104.196.6.119

In [62]:
import mysql.connector
import pandas as pd

# Define your database connection
connection = mysql.connector.connect(
    host="horseprofileshub.czyece6mq0kn.eu-north-1.rds.amazonaws.com",
    user="abiqb93",
    password="Saps123$#",
    database="horseprofileshub",
    port=3306
)

try:
    # Create a cursor
    cursor = connection.cursor(dictionary=True)

    # Define the query to fetch records from 2020 onwards
    query = """
        SELECT *
        FROM APIData_Table2
        WHERE meetingDate >= '2024-01-01'
    """

    # Execute the query
    cursor.execute(query)

    # Fetch all results
    results = cursor.fetchall()

    # Convert results to a pandas DataFrame
    df = pd.DataFrame(results)

    # Display a preview
    print("Retrieved records:")
    print(df.head())

finally:
    cursor.close()
    connection.close()


Retrieved records:
   positionOfficial           horseName countryCode        sireName  \
0                 1  IT'S ALL ABOUT YOU         GBR   HIGHLAND REEL   
1                 3    DANCE AND GLANCE         GBR  PASSING GLANCE   
2                 2      NORAH'S LEGACY         GBR           YEATS   
3                 4   FANCY NANCY PANTS         GBR     DYLAN MOUTH   
4                 5  SHADOWFAX OF ROHAN         GBR            None   

        damName raceSurfaceName raceType distance  \
0   THREE MOONS            Turf   Hurdle     19.3   
1  CYD CHARISSE            Turf   Hurdle     19.3   
2    LADY CHLOE            Turf   Hurdle     19.3   
3  BESPOKE LADY            Turf   Hurdle     19.3   
4          None            Turf   Hurdle     19.3   

                            raceTitle   going  ...          meetingDate  \
0  HAPPY NEW YEAR NOVICES' HURDLE (4)  Gd/Sft  ...  2025-01-01 00:00:00   
1  HAPPY NEW YEAR NOVICES' HURDLE (4)  Gd/Sft  ...  2025-01-01 00:00:00   
2  HAPPY N

In [63]:
cols_to_numeric = [
    'distance',
    'distanceBeaten',
    'performanceRating',
    'timefigure',
    'prizeMoneyWon'
]

for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [64]:
df.performanceSymbol.unique()

array(['+', None, '?'], dtype=object)

In [65]:
df = df[df['countryCode'] != 'SAF']

In [66]:
df = df[df['raceType'] == 'Flat']

In [67]:
import pandas as pd

# Parse dates
df['meetingDate'] = pd.to_datetime(df['meetingDate'], format='mixed')

# Filter from 2024 onwards
df = df[df['meetingDate'] > pd.Timestamp('2020-01-01')].copy()

# Ensure rating is numeric
df['performanceRating'] = pd.to_numeric(df['performanceRating'], errors='coerce')

# Sort
df = df.sort_values(by=['horseName', 'meetingDate'])

# Get most recent 4 meetingDates
recent_dates = df['meetingDate'].dropna().sort_values(ascending=False).unique()[:4]

# Get horses that ran in last 3 meetings
recent_horses = df[df['meetingDate'].isin(recent_dates)]['horseName'].unique()
df_recent = df[df['horseName'].isin(recent_horses)].copy()

# Initialize condition flags
df_recent['condition1'] = 0
df_recent['condition2'] = 0
df_recent['condition3'] = 0

# Evaluation function
def evaluate_conditions(group):
    group = group.sort_values(by='meetingDate', ascending=False).reset_index(drop=True)

    # Only apply Condition 1 if the horse has 3 or fewer races
    if len(group) <= 3:
        group.loc[:2, 'condition1'] = group.loc[:2, 'performanceRating'] > 80

    # Condition 2: Rating > 80 and >5 points higher than all previous ratings
    group['previous_max'] = group['performanceRating'].expanding().max().shift(1)
    group['rating_diff_from_prev_max'] = group['performanceRating'] - group['previous_max']
    group.loc[(group['performanceRating'] > 80) &
              (group['rating_diff_from_prev_max'] > 5), 'condition2'] = 1


    # Condition 3: Last race (most recent) contains 'p' in performanceSymbol
    latest_row = group.iloc[0]
    if isinstance(latest_row['performanceSymbol'], str) and 'p' in latest_row['performanceSymbol'].lower():
        group.loc[0, 'condition3'] = 1

    return group.sort_values(by='meetingDate')  # return in original order

# Apply per horse
df_result = df_recent.groupby('horseName', group_keys=False).apply(evaluate_conditions)

# Get latest race per horse
latest_per_horse = df_result.sort_values(by='meetingDate').groupby('horseName').tail(1)

# Final report
report = latest_per_horse[['horseName', 'condition1', 'condition2', 'condition3']]
report[['condition1', 'condition2', 'condition3']] = report[['condition1', 'condition2', 'condition3']].astype(int)

# Filter horses matching any condition
report = report[(report['condition1'] == 1) |
                (report['condition2'] == 1) |
                (report['condition3'] == 1)]

# Output
print(report.sort_values(by='horseName').reset_index(drop=True))


/tmp/ipython-input-1317098587.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.loc[:2, 'condition1'] = group.loc[:2, 'performanceRating'] > 80
/tmp/ipython-input-1317098587.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False False False]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.loc[:2, 'condition1'] = group.loc[:2, 'performanceRating'] > 80
/tmp/ipython-input-1317098587.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.loc[:2, 'condition1'] = group.loc[:2, 'performanceRating'] > 80
/tmp/ipython-input-

             horseName  condition1  condition2  condition3
0            ADVANCING           1           0           0
1           BEN LAWERS           1           0           0
2     BETTE DAVIS EYES           1           0           0
3              CADOGAN           1           0           0
4              DANDANA           1           0           0
5            DAYDREAMA           1           0           0
6    EARL OF ROCHESTER           1           0           0
7      FIRST PRINCIPLE           1           0           0
8     FOREVER PENYWERN           1           0           0
9               GOYARD           1           0           0
10         GREAT MATES           1           0           0
11           GRIZEDALE           1           0           0
12          JEL PEPPER           1           0           0
13            JOE STAR           1           0           0
14              LAFFAN           1           0           0
15               LOOKA           1           0          

/tmp/ipython-input-1317098587.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df_recent.groupby('horseName', group_keys=False).apply(evaluate_conditions)
/tmp/ipython-input-1317098587.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report[['condition1', 'condition2', 'condition3']] = report[['condition1', 'condition2', 'condition3']].astype(int)


In [68]:
report = report[(report['condition1'] == 1) |
                (report['condition2'] == 1) |
                (report['condition3'] == 1)]



In [69]:
import mysql.connector


# Step 1: Connect to your MySQL database
connection = mysql.connector.connect(
    host="horseprofileshub.czyece6mq0kn.eu-north-1.rds.amazonaws.com",
    user="abiqb93",
    password="Saps123$#",
    database="horseprofileshub",
    port=3306
)

cursor = connection.cursor()

# Step 2: Drop the table if it exists
drop_table_query = """
DROP TABLE IF EXISTS review_horses;
"""

# Step 3: Create the table
create_table_query = """
CREATE TABLE review_horses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    horseName VARCHAR(255),
    condition1 TINYINT,
    condition2 TINYINT,
    condition3 TINYINT,
    notes TEXT
);
"""

# Step 4: Insert data from the filtered report
insert_query = """
INSERT INTO review_horses (horseName, condition1, condition2, condition3, notes)
VALUES (%s, %s, %s, %s, %s);
"""

try:
    # Execute table drop and create
    cursor.execute(drop_table_query)
    print("🗑️  Dropped table `review_horses` (if it existed).")

    cursor.execute(create_table_query)
    print("✅ Created table `review_horses`.")

    # Prepare the data for insertion
    data_to_insert = [
        (row['horseName'], int(row['condition1']), int(row['condition2']), int(row['condition3']), "")
        for _, row in report.iterrows()
    ]

    # Execute bulk insert
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()

    print(f"✅ Inserted {cursor.rowcount} rows into `review_horses`.")

except mysql.connector.Error as err:
    print(f"❌ Error: {err}")

# Step 5: Close the connection
cursor.close()
connection.close()
print("🔒 Connection closed.")


🗑️  Dropped table `review_horses` (if it existed).
✅ Created table `review_horses`.
✅ Inserted 35 rows into `review_horses`.
🔒 Connection closed.
